In [ ]:
#!/usr/bin/env python
# coding: utf-8
# In[1]:
#!/usr/bin/env python
# coding: utf-8
# get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import time
import numpy as np
from datetime import datetime
from sklearn.externals import joblib 
import os
import glob
from konlpy.tag import Mecab
import lightgbm as lgb
print(lgb.__version__)
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib 
from sklearn.model_selection import StratifiedKFold, KFold
import gc
from tqdm import tqdm_notebook, tqdm
import json
from typing import NamedTuple
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings(action='ignore')
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
print(torch.__version__)
# from tools import eval_summary, save_feature_importance, merge_preds
from tools import EarlyStopping

device = torch.device('cpu')
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# In[2]:
torch.set_num_threads(8)
torch.get_num_threads()

#### Load Data

In [ ]:
df_train = pd.read_csv('input/train.csv', dtype=np.float32)
df_test = pd.read_csv('input/test.csv', dtype=np.float32)
print(df_train.shape, df_test.shape)


In [ ]:
layer_cols = [c for c in df_train.columns if 'layer_' in c]
fea_cols = [c for c in df_train.columns if c not in layer_cols]

len(fea_cols), len(layer_cols)

In [ ]:
df_model = df_train[:8100]

#### Model

##### DCNNModel

In [ ]:
# # size = 48
# W = 15 # input_volume_size
# F = 6  # kernel_size
# S = 1   # strides
# P = 1
# # padding_size

# size = (W - F + 2*P) / S + 1
# size
# # ((size - 1) * S) - 2*P + F

In [ ]:
class DCNNModel(torch.nn.Module):
    def __init__(self, input_size, dropout_probability=0.3):
        super(DCNNModel,self).__init__()
#         relu = torch.nn.ReLU()
        act = torch.nn.ELU()
        dropout = torch.nn.Dropout(p=dropout_probability)

        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 1024), torch.nn.BatchNorm1d(1024), act, dropout, 
            torch.nn.Linear(1024, 1024), torch.nn.BatchNorm1d(1024), act, dropout,            
            torch.nn.Linear(1024, 512), torch.nn.BatchNorm1d(512), act, dropout,
            torch.nn.Linear(512, 512), torch.nn.BatchNorm1d(512), act, dropout,
            torch.nn.Linear(512, 256), torch.nn.BatchNorm1d(256), act, dropout,            
            torch.nn.Linear(256, 128),
            
        )
        
        self.cnn = torch.nn.Sequential(
            torch.nn.Conv1d(1, 12, 3, stride=1, padding=1), torch.nn.BatchNorm1d(12), act,
            torch.nn.Conv1d(12, 12, 3, stride=1, padding=1), torch.nn.BatchNorm1d(12), act,
            torch.nn.MaxPool1d(2),
            torch.nn.Conv1d(12, 24, 3, stride=1, padding=1), torch.nn.BatchNorm1d(24), act,
            torch.nn.Conv1d(24, 24, 3, stride=1, padding=1), torch.nn.BatchNorm1d(24), act,
            torch.nn.MaxPool1d(2),
            torch.nn.Conv1d(24, 48, 3, stride=1, padding=1), torch.nn.BatchNorm1d(48), act,
            torch.nn.Conv1d(48, 48, 3, stride=1, padding=1), torch.nn.BatchNorm1d(48), act,
            torch.nn.MaxPool1d(2),
            torch.nn.Conv1d(48, 96, 3, stride=1, padding=1), torch.nn.BatchNorm1d(96), act,
            torch.nn.Conv1d(96, 96, 3, stride=1, padding=1), torch.nn.BatchNorm1d(96), act,
            torch.nn.MaxPool1d(2),
            torch.nn.Conv1d(96, 96, 3, stride=1, padding=1), torch.nn.BatchNorm1d(96), act,
            torch.nn.Conv1d(96, 96, 3, stride=1, padding=1), torch.nn.BatchNorm1d(96), act,
            torch.nn.MaxPool1d(2),
        )
        
        self.clf = torch.nn.Sequential(
            torch.nn.Linear(128 + 672, 512), act, dropout,
            torch.nn.Linear(512, 512), act, dropout,
            torch.nn.Linear(512, 4),
        )
        
    def forward(self, x):
        out_cnn = self.cnn(x.unsqueeze(1))
        dim = 1
        for d in out_cnn.size()[1:]:
            dim = dim * d
        out_cnn = out_cnn.view(-1, dim)
        
        out = self.clf(torch.cat([self.model(x), out_cnn], axis=1))
        return out
        
 

#### DataSet

In [ ]:
   
class SemiDataset(Dataset):
    def __init__(self, df, fea_cols, y_cols):        
        self.X = df[fea_cols].values
        self.y = df[y_cols].values
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx].astype(np.float32), self.y[idx].astype(np.float32)
    

#### Trainer

In [ ]:
class Trainer(object):
    def __init__(self, model, criterion, optimizer, scheduler, device):
        self.device = device
        self.model = model#.to(self.device)
        self.criterion = criterion#.to(self.device)
        self.optimizer = optimizer
        self.scheduler = scheduler

        print(self.model.train())
        pass
    
    def train(self, data_loader):
        self.model.train()
        total_loss = 0
        for data in data_loader:
            X_batch, y_batch = data
            X_batch = X_batch.to(self.device)
            y_batch = y_batch.to(self.device)
            
            y_pred = self.model(X_batch)
#             print(y_pred, y_batch)
            
            loss = self.criterion(y_pred, y_batch)
            total_loss = total_loss + loss.item()

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        self.scheduler.step()
        
        return total_loss / len(data_loader)
    
    def eval(self, data_loader):
        self.model.eval()
        total_loss = 0
#         print('valid_loader', len(valid_loader))
        for data in data_loader:
            X_batch, y_batch = data
            X_batch = X_batch.to(self.device)
            y_batch = y_batch.to(self.device)
            with torch.no_grad():
                y_pred = self.model(X_batch)
                loss = self.criterion(y_pred, y_batch)
                total_loss = total_loss + loss.item()
        return total_loss / len(data_loader)

    def save(self, model_path='checkpoint.pt'):
        torch.save(self.model.state_dict(), 'checkpoint.pt')
        return
    
    def load(self, model_path='checkpoint.pt'):
        self.model.load_state_dict(torch.load(model_path))
        return

#### Train

In [ ]:
model_ts = datetime.now().strftime('%Y%m%dT%H%M%S')
print(model_ts)

print(f'fea_size {len(fea_cols)} layer_cols {layer_cols}')

In [ ]:
torch.manual_seed(81511991154)
torch.initial_seed()

In [ ]:
dataset = SemiDataset(df_model[fea_cols + layer_cols], fea_cols, layer_cols)

train_set, val_set = torch.utils.data.random_split(dataset, [7000, 1100])

print(len(train_set), len(val_set))

batch_size = 50000
num_workers = 4

loader_params = {    
    'batch_size' : batch_size,
    'shuffle' : True,
    'num_workers' : num_workers,
    'drop_last' : False,
}
train_loader = DataLoader(dataset=train_set, **loader_params)
val_loader = DataLoader(dataset=val_set, **loader_params)

print(f'batch_size {batch_size} num_workers {num_workers}')
print(f'train_loader {len(train_loader)} val_loader {len(val_loader)}')

In [ ]:
model = DCNNModel(input_size=len(fea_cols), dropout_probability=0.01).to(device)


In [ ]:
# torch.load('model/20200123T110746_1.2787836790084839.model')

In [ ]:
    
criterion = nn.L1Loss(reduction='mean').to(device)
# criterion = nn.MSELoss(reduction='mean').to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
# optimizer = torch.optim.Adam(model.parameters(), lr = 0.00005)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

scheduler = StepLR(optimizer, step_size=200, gamma=0.1)


trainer = Trainer(model, criterion, optimizer, scheduler, device)
# trainer.load()
scheduler.get_lr()[0]

In [ ]:
early_stopping = EarlyStopping(patience=100, min_epoch=10, verbose=True)

total_epoch = 2000

for e in tqdm_notebook(range(total_epoch), total=total_epoch, desc='Epoch'):
    if os.path.isfile('stop.flag'):
        print(f'{e} stop!')
        break

    train_loss = trainer.train(train_loader)
    valid_loss = trainer.eval(val_loader)
    
    ts = datetime.now().strftime('%Y%m%dT%H%M%S')
    print(f'[{ts}] Epock {e} / {total_epoch}\t lr {scheduler.get_lr()[0]}')
    print(f'  train_loss: {train_loss}  valid_loss: {valid_loss}')
    
    early_stopping(valid_loss, model)

    if early_stopping.early_stop:
        print("\tEarly stopping epoch {}, valid loss {}".format(e, early_stopping.val_loss_min))
        break
        
        

model.load_state_dict(torch.load('model/checkpoint.pt'))
torch.save(model.state_dict(), 'model/{}_{}.model'.format(model_ts, early_stopping.val_loss_min))
print('model/{}_{}.model'.format(model_ts, early_stopping.val_loss_min))
    
# torch.save(model.state_dict(), f'checkpoint.pt.{train_loss}')

In [ ]:
model.eval()
y_pred = model(torch.Tensor(df_test[fea_cols].values).to(device))    
print(y_pred, y_pred.round())

In [ ]:
ts = datetime.now().strftime('%Y%m%dT%H%M%S')

df_submit = pd.read_csv('input/sample_submission.csv', index_col=0)

df_submit[layer_cols] = y_pred.cpu().detach().numpy()
df_submit.to_csv(f'submit/{ts}_{early_stopping.val_loss_min}_.csv')

df_submit[layer_cols] = y_pred.round().cpu().detach().numpy()
df_submit.to_csv(f'submit/{ts}_{early_stopping.val_loss_min}.csv')

print(ts, early_stopping.val_loss_min)